## Huspris

Her har vi data som beskriver boligpriser i Ames, Iowa fra 2006 til 2010. Datasettet inneholder mange variabler som kan brukes til å vurdere boligverdi. Se [her](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview) for mer informasjon om data. I denne oppgaven ser vi kun på de numeriske data. 

In [ ]:
# imports
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# les inn husprisdata
df = pd.read_csv('data/huspris.csv')

In [ ]:
# del data i mål-, og prediktorvariabler
reduced_df = df.select_dtypes(include=[np.number]).drop('Id', axis=1).dropna()
X = reduced_df.drop('SalePrice', axis=1).values
y = reduced_df['SalePrice'].values

Del data i trenings-, validerigns-, og testdata med størrelser 70%, 15%, 15% av data. 

In [ ]:
# dele data i trenings, validerings og testdata
# generer X_train, X_val, X_test, y_train, y_val, y_test

# vi gjør at 70% blir treningsdata

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3
)

X_val, X_test, y_val, y_test = train_test_split(
    X_val, y_val, test_size=0.5
)



Tren en Grunnlinjemodell på treningsdata og finn validerings-RMSE. 

In [ ]:

baseline = DummyRegressor(strategy="mean")
baseline.fit(X_test, y_test)
baseline

# mse error given the prdicted line, what is distance from the line to actual point, squared and then the mean of all of these
y_predicted = baseline.predict(X_val) #given a new data set x, predict the y values
rmse_baseline = np.round(np.sqrt(mean_squared_error(y_val, y_predicted, squared=False))) #for each predicted y, check distance from known y
rmse_baseline

Tren en Lasso regresjonsmodell (sklearn.linear_model.Lasso) med hyperparameter alpha mellom 1 og 500 på treningsdata. Sorter de ulike modellene etter mean kvadrert feil på valideringsdata (sklearn.metrics.mean_squared_error).
Visualiser hvordan mean kvadratisk feil avhenger av alpha. 

In [ ]:
# tren forskjellige modeller

alpha_values = []
mse_values = []

for alpha in np.arange(1, 500, 10):

    clf = Lasso(alpha=alpha)
    clf.fit(X_train, y_train)

    y_predicted = clf.predict(X_val)
    
    # sjekk MSE for valideringsdata
    mse = mean_squared_error(y_val, y_predicted, squared=True)

    alpha_values.append(alpha)
    mse_values.append(mse)

import matplotlib.pyplot as plt

# visualiser validerings-MSE avhengig av alpha

plt.plot(alpha_values,mse_values)
plt.xlabel("alpha_values")
plt.ylabel("mse_values")
plt.show()


## Visualization comment
Here one can see that as alpha goes towards ca 300 MSE decreases, and increases after ca 375. In this case, a alpha value of ~325 is best as the average MSE is better.

Lag alle polynomkombinasjoner av grad 2 av data (sklearn.preprocessing.PolynomialFeatures). 
Tren en Lasso regresjonsmodell (sklearn.linear_model.Lasso) med hyperparameter alpha mellom 500 og 1500 på polynomkombinasjoner av treningsdata.
Sorter de ulike modellene etter mean kvadrert feil på valideringsdata (sklearn.metrics.mean_squared_error).
Visualiser hvordan mean kvadratisk feil avhenger av alpha. 

In [ ]:
# lag datasett med polynomielle data

poly = PolynomialFeatures(degree=2)

X_train_pf = poly.fit_transform(X_train)
X_val_pf = poly.transform(X_val)
X_test_pf = poly.transform(X_test)

In [ ]:
# tren forskjellige modeller

alpha_values_poly = []
mse_values_poly = []
pf_models = []

for alpha in np.arange(500, 1500, 50): # hadde jump = 10 men den tok sykt langt tid
    clf = Lasso(alpha=alpha)
    clf.fit(X_train_pf, y_train) 
 
    y_predicted = clf.predict(X_val_pf)
    
    #finn mse
    pf_mse = mean_squared_error(y_val, y_predicted, squared=True)

    alpha_values_poly.append(alpha)
    mse_values_poly.append(pf_mse)
    pf_models.append(clf)
    
pf_models


In [ ]:
# visualiser validerings-MSE avhengig av alpha
plt.plot(alpha_values_poly,mse_values_poly)
plt.xlabel("alpha_values")
plt.ylabel("mse_values")
plt.show()


### Comment:

In this case we can see that alpha value and mse value are correlated.  It is clear that the higher the alpha value, the lower the MSE.

Se om du kan finne en bedre modell med en annen metode enn Lasso regresjon (e.g. sklearn.linear_model.ElasticNet, sklearn.ensemble.RandomForestRegressor, sklearn.svm.SVR, sklearn.gaussian_process.GaussianProcessRegressor). Finn gode hyperparametre til metoden du velger ut. 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor

# tren forskjellige modeller

#her prøver jeg på litt fancy greie ved å kun lage settings ett sted.
#det er kult, men kanskje litt vel overkomplisert

models = [
    {"model_type": Lasso, "settings": {"alpha": 1500}},
    {"model_type": RandomForestRegressor, "settings": {"n_estimators":100}}, #hadde n_estimators på 1000, da funker an bra men tar sykt lang tid
    {"model_type": ElasticNet, "settings": {"alpha": 1500}}, 
    {"model_type": SVR, "settings": {"degree":2}},
    {"model_type": GaussianProcessRegressor, "settings": {"alpha": 1500}},
]

model_strings = []
mse_values_models = []

for mod in models:
    clf = mod["model_type"](**mod["settings"]) #henter ut settings her
    clf.fit(X_train_pf, y_train) 

    y_predicted = clf.predict(X_val_pf)

    #finn mse
    pf_mse = mean_squared_error(y_val, y_predicted, squared=True)

    mse_values_models.append(pf_mse)
    model_strings.append(str(mod["model_type"].__name__))



data_models = pd.DataFrame({
    'model_name': model_strings,
    'mse_values': mse_values_models
})

fig = px.bar(data_models, x='model_name', y='mse_values', 
             title='MSE values for different models', 
             labels={'x':'Model', 'y':'Mean Squared Error'})

fig.show()
print(data_models)


In [ ]:
# sjekk MSE for valideringsdata
data_models.sort_values(by='mse_values', inplace=True)
mse = data_models
print(mse)

Velg ut den beste modellen og sjekk hvor godt den generaliserer ved å regne ut mean kvadrert feil og kvadratrooten av mean kvadrert feil på testdata. 

In [ ]:
# sjekk generaliseringsevne
from math import sqrt


best_model = RandomForestRegressor(n_estimators=100)
best_model.fit(X_train_pf, y_train) 

y_test_predicted = best_model.predict(X_test_pf)

test_mse = mean_squared_error(y_test, y_test_predicted)

test_rmse = sqrt(test_mse)

print('Test MSE:', test_mse)
print('Test RMSE:', test_rmse)

test_mse

Gi en oppsummering over hva du har gjort og hva resultatet var. 

Lest inn husprisdata, delt i trening,test og validation

Sett litt på en grunnlinjemodell og dens RMSE

trent lasso modell og sett hvordan mse endrer seg når alpha endrer seg

alpha opp = mse ned

trent noen flere lasso modeller og sett på validerings data mse, når vi endrer alpha

så har vi trent noen flere modeller og sett hvilken som har lavest MSE og det var ```RandomForestRegressor```

Så har vi sjekket ut den modellen, sett på hvor god den er på testdataen. Det viser seg at mse er  høy, så kanskje denne modellen ikke generaliser så bra. 

Validation MSE: 747310700, test MSE: 1381201492. Increase = 180%

RMSE er grei, men sammenlignet med validation data, ser vi at MSE er litt for høy. Det er snakk om huspris data så vi forventer høye verdier, men dette er for høyt. Jeg tror at modellen er overfitted på testdata. 